## Loading of Miller ECoG data of the memory nback task

includes some visualizations

In [1]:
#@title Data retrieval
import os, requests

fname = 'memory_nback.npz'
url = "https://osf.io/ez4hs/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)


In [2]:
#@title Import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [3]:
#@title Data loading
import numpy as np

alldat = np.load(fname, allow_pickle=True)['dat']

# Select just one of the recordings here. This is subject 1, block 1.
dat = alldat[1][1]

print(dat.keys())


dict_keys(['V', 't_off', 'srate', 't_on', 'target', 'stim_id', 'response', 'rt', 'expinfo'])


# Dataset info #

This is one of multiple ECoG datasets from Miller 2019, recorded in clinical settings with a variety of tasks. Raw data here:

https://exhibits.stanford.edu/data/catalog/zk881ps0522

`dat` contain 3 sessions from 3 subjects, and was not yet used in a publication, so it's fresh data for you to analyze!  

Each subject's data is divided into 5 sessions: fixation, exp1, exp2, exp3, fixation, which are consecutive blocks in the data structure (i.e. `alldat[0][0]`, `alldat[0][1]`, ..., `alldat[0][4]` for subject 0). Exp1 is a "0-back" memory experiment, where the subjects have to identify the picture of one of the houses (stimid = 10), which they have memorized at the beginning of the stimulus block. Exp2 is a 1-back memory experiment, where the participants have to respond to images of repeated houses. Finally, exp3 is 2-back where participants respond to pictures of houses that are repeated after another intervening random picture. This task was hard for these participants and most of them did not really respond at all, but the data can still be used to ask questions about overall shifts of neural activity in a hard task when the subjects feel overwhelmed by the memory requirements.

Note that for the third/last subject, no response data was collected, so only the first two subjects can be analyzed for their correct / incorrect trials.   

Sample rate is always 1000Hz, and the ECoG data has been notch-filtered at 60, 120, 180, 240 and 250Hz, followed by z-scoring across the entire recording and conversion to float16 to minimize size. 

Variables for each block within each subject: 
* `dat['V']`: continuous voltage data (time by channels)
* `dat['expinfo']`: experiment type for this block
* `dat['srate']`: sampling rate for voltage data, always 1000Hz
* `dat['t_on']`: time of stimulus onset in data samples
* `dat['t_off']`: time of stimulus offset, usually 600 samples after `t_on`
* `dat['stim_id`]: identity of house stimulus from 1 to 40. Stimulus 10 was the target in the 0-back task.  
* `dat['target']`: 0 or 1, indicates if this stimulus requires a response
* `dat['response']`: 0 or 1, indicates if the subject actually made a response
* `dat['rt']`: reaction time for trials with a response in voltage data samples (1000Hz). 


In [4]:
# divide into trials and average
dat = alldat[1][1]
V = dat['V']

nt, nchan = V.shape
nstim = len(dat['t_on'])

# use a timerange from 400ms before to 1600ms after the stimulus onset
trange = np.arange(-400, 1600)
ts = dat['t_on'][:,np.newaxis] + trange
V_epochs = np.reshape(V[ts, :], (nstim, 2000, nchan))

V_resp = (V_epochs[dat['response']==1]).mean(0)
V_base  = (V_epochs[dat['response']==0]).mean(0)

In [ ]:
# let's find the electrodes that distinguish responses from non-responses
# note these are event-related potentials
from matplotlib import pyplot as plt

plt.figure(figsize=(15,12))
for j in range(60):
  ax = plt.subplot(6,10,j+1)
  plt.plot(trange, V_resp[:,j], lw=2)
  plt.plot(trange, V_base[:,j], lw=2)
  plt.title('ch%d'%j)
  plt.ylim([-2, 2])

In [ ]:
# let's look at all the response trials for the electrode that has a positive ERP
plt.subplot(1,3,1)
plt.imshow(V_epochs[dat['response']==1,:,2].astype('float32'), aspect='auto')
plt.colorbar()

In [ ]:
# let's look at all the response trials for an electrode that has a positive ERP
plt.subplot(1,3,1)
plt.imshow(V_epochs[dat['response']==1,:,19].astype('float32'), aspect='auto')
plt.colorbar()

In [8]:
# to do: do these responses covary with the reaction times? are they present on trials where there is no motor response? 